<a href="https://colab.research.google.com/github/bofeng2268/qm2/blob/main/random_routes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tabulate

In [19]:
import random

# function to generated a list of 100 random coordinates to use as starting points for routes to Royal London Hospital
def generate_random_coordinates(num_points, central_lat_lon, greater_lat_lon):
    random_coords = []
    while len(random_coords) < num_points:
        lat = random.uniform(greater_lat_lon['min_lat'], greater_lat_lon['max_lat'])
        lon = random.uniform(greater_lat_lon['min_lon'], greater_lat_lon['max_lon'])

        if central_lat_lon['min_lat'] <= lat <= central_lat_lon['max_lat'] and central_lat_lon['min_lon'] <= lon <= central_lat_lon['max_lon']:
            continue
        random_coords.append((lat, lon))

    return random_coords

# defined the boundary coordinates for Central London and Greater London
central_london = {'min_lat': 51.44, 'max_lat': 51.56, 'min_lon': -0.18, 'max_lon': 0.02}
greater_london = {'min_lat': 51.38, 'max_lat': 51.60, 'min_lon': -0.33, 'max_lon': 0.15}

# generated list using function
random_coordinates = generate_random_coordinates(100, central_london, greater_london)

In [20]:
import geopandas as gpd
import plotly.express as px
import pandas as pd

# loading London topojson
london_shapefile = gpd.read_file('https://gist.githubusercontent.com/cejast/2cb80a2346b2049ac5d0/raw/c7ea98c7b9204782b6652b29bf3ef3e3b1a187ea/london-topojson.json')

# extracting latitude and longitude from the list of coordinates
latitudes, longitudes = zip(*random_coordinates)

# creating a datframe with extracted coordinates
df = pd.DataFrame({'Latitude': latitudes, 'Longitude': longitudes})

# creating a scatterplot on a Mapbox
fig = px.scatter_mapbox(df, lat='Latitude', lon='Longitude',
                        hover_name='Latitude', hover_data=['Longitude'],
                        title='Random Coordinates on Greater London Map',
                        mapbox_style="carto-positron",
                        center={"lat": 51.5074, "lon": -0.1278},
                        zoom=10)

fig.update_layout(margin={"r":0, "t":0, "l":0, "b":0})

# creating a dataframe for the Royal London Hospital coordinates
hospital_df = pd.DataFrame({
    'Latitude': [51.517327],
    'Longitude': [-0.059259]
})

# adding the hospital location as a separate scatter plot layer
fig.add_trace(
    px.scatter_mapbox(hospital_df, lat='Latitude', lon='Longitude',
                      color_discrete_sequence=['red']).data[0]
)

fig.show()


In [21]:
import requests
import json
import pandas as pd
import random
from tabulate import tabulate

#function to parse json responses of TFL API requests
def parse_journey_data(journey_data):
    if 'journeys' not in journey_data or not journey_data['journeys']: # requests had some missing data depending on the routes, so checking and giving None output when necessary
        return None, None

    shortest_duration = min(journey_data['journeys'], key=lambda x: x['duration'])['duration'] #finding minimum duration
    shortest_journey = next(j for j in journey_data['journeys'] if j['duration'] == shortest_duration) #finds journey that has thee shortest duration that we previously found
    transport_changes = len(shortest_journey['legs']) #number of times one has to change transport during journey called "legs"

    return shortest_duration, transport_changes

# dataframe to store journey data for disabled and non-disabled people for each route
journey_data_df = pd.DataFrame(columns=['Start Coordinate', 'Disabled Duration', 'Disabled Changes',
                                        'Non-Disabled Duration', 'Non-Disabled Changes'])

api_key = 'd963cc094f604b6bbb599c4602b8d3a7' #could also import it from private file, but is fine as i used it only for the purpose of this code, not confidential

for start_point in random_coordinates:

    start_point = f"{start_point[0]}, {start_point[1]}"  #respective random points coordinates
    end_point = '51.517327, -0.059259'  #Royal London Hospital coordinates
    url = f'https://api.tfl.gov.uk/Journey/JourneyResults/{start_point}/to/{end_point}'

    headers = {
        'app_key': api_key,
    }

    # request parameters for a disabled person with full step-free access and only using tube
    params_disabled = {
        'accessibilityPreference': 'StepFreeToVehicle,StepFreeToPlatform',
        'mode': 'tube'
    }

    # request parameters for a non-disabled person using only tube
    params_non_disabled = {
        'mode': 'tube'
    }

    response_disabled = requests.get(url, headers=headers, params=params_disabled)
    journey_disabled = response_disabled.json()

    response_non_disabled = requests.get(url, headers=headers, params=params_non_disabled)
    journey_non_disabled = response_non_disabled.json()

    shortest_duration_disabled, transport_changes_disabled = parse_journey_data(journey_disabled)# disabled journey data (duration and trasport changes)
    shortest_duration_non_disabled, transport_changes_non_disabled = parse_journey_data(journey_non_disabled) # non-disabled journey data (duration and trasport changes)

    new_row = pd.DataFrame([{
    'Start Coordinate': start_point,
    'Disabled Duration': shortest_duration_disabled,
    'Disabled Changes': transport_changes_disabled,
    'Non-Disabled Duration': shortest_duration_non_disabled,
    'Non-Disabled Changes': transport_changes_non_disabled
    }])
    journey_data_df = pd.concat([journey_data_df, new_row], ignore_index=True) #adding new row with respective columns for each jouney to the dataframe

journey_data_df = journey_data_df.dropna() # dropping rows that have None values

print(tabulate(journey_data_df, headers='keys', tablefmt='pretty')) #printing for visualisation



+----+------------------------------------------+-------------------+------------------+-----------------------+----------------------+
|    |             Start Coordinate             | Disabled Duration | Disabled Changes | Non-Disabled Duration | Non-Disabled Changes |
+----+------------------------------------------+-------------------+------------------+-----------------------+----------------------+
| 1  | 51.39350931144743, -0.22775204117425796  |        121        |        4         |          94           |          5           |
| 3  |  51.55034257266594, 0.09318587473067008  |        60         |        3         |          57           |          3           |
| 6  | 51.434043992902545, -0.1976123503433892  |        79         |        4         |          59           |          3           |
| 10 |  51.580068310718175, 0.0996317936585282  |        141        |        6         |          52           |          4           |
| 13 | 51.556565736459284, -0.24699084494840834 